In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import jaccard_score

# Function to load and preprocess images from a directory
def load_and_preprocess_images_from_folder(folder, target_shape=(256, 256)):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, target_shape)
            images.append(img)
    return np.array(images)

# Load images and labels from respective folders
images_folder = '/kaggle/input/astroid-hunter-dataset/ast/Original Images'
labels_folder = '/kaggle/input/astroid-hunter-dataset/ast/Binary Data'

X = load_and_preprocess_images_from_folder(images_folder)
y = load_and_preprocess_images_from_folder(labels_folder)

# Convert multi-class labels to a single binary channel (foreground vs. background)
y_binary = np.any(y, axis=-1).astype(np.float32)

# One-hot encode the labels
y_binary_onehot = tf.keras.utils.to_categorical(y_binary, num_classes=2)

# Preprocessing
X = X.astype('float32') / 255.0

# Split dataset into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary_onehot, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Standard U-Net Model
def unet(input_size=(256, 256, 3)):
    inputs = Input(input_size)
    
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(Conv2DTranspose(512, 2, strides=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)
    
    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(Conv2DTranspose(256, 2, strides=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)
    
    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(Conv2DTranspose(128, 2, strides=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)
    
    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(Conv2DTranspose(64, 2, strides=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    
    conv10 = Conv2D(2, 1, activation='softmax')(conv9)
    
    model = Model(inputs=inputs, outputs=conv10)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Create and compile the U-Net model
model = unet(input_size=(256, 256, 3))

# Train the model with data augmentation
batch_size = 8
epochs = 10

train_datagen = datagen.flow(X_train, y_train, batch_size=batch_size)
val_datagen = ImageDataGenerator().flow(X_val, y_val, batch_size=batch_size)

steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_val) // batch_size

model.fit(train_datagen, steps_per_epoch=steps_per_epoch, epochs=epochs, validation_data=val_datagen, validation_steps=validation_steps)

# Evaluate the model on test data
y_pred = model.predict(X_test)
y_pred_binary = np.argmax(y_pred, axis=-1)

accuracy = accuracy_score(np.argmax(y_test, axis=-1).flatten(), y_pred_binary.flatten())
precision = precision_score(np.argmax(y_test, axis=-1).flatten(), y_pred_binary.flatten())
recall = recall_score(np.argmax(y_test, axis=-1).flatten(), y_pred_binary.flatten())
f1 = f1_score(np.argmax(y_test, axis=-1).flatten(), y_pred_binary.flatten())
conf_matrix = confusion_matrix(np.argmax(y_test, axis=-1).flatten(), y_pred_binary.flatten())
iou = jaccard_score(np.argmax(y_test, axis=-1).flatten(), y_pred_binary.flatten())

print("Testing Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)
print("IOU:", iou)

# Generate segmentation results on sample images from the testing set
num_samples = 5
sample_indices = np.random.choice(len(X_test), num_samples, replace=False)

for idx in sample_indices:
    img = X_test[idx]
    mask_true = y_test[idx]
    mask_pred = y_pred[idx]

    # Visualize the original image
    cv2.imshow("Original Image", img)
    cv2.waitKey(0)

    # Visualize true mask
    cv2.imshow("True Mask", mask_true)
    cv2.waitKey(0)

    # Visualize predicted mask
    cv2.imshow("Predicted Mask", mask_pred)
    cv2.waitKey(0)

cv2.destroyAllWindows()
